## Evaluating regression techniques for speaker characterization - Part II
### Laura Fernández Gallardo

In this notebook, I will evaluate the performance of different regression techniques for characterizing the user, given the data explored in Part I.

In [ ]:
# fix random seed for reproducibility
seed = 2302
np.random.seed(seed)